In [3]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [4]:
contract_url = 'https://www.basketball-reference.com/contracts/players.html'

In [9]:
def scrape_contract_data():
    r = requests.get(contract_url)
    soup = BeautifulSoup(r.text, 'html.parser')

    for header in soup.find_all(class_='over_header'):
        header.decompose()

    for thead in soup.find_all(class_='thead'):
        thead.decompose()

    player_links = []
        
    for a in soup.find_all('td', {'data-stat': 'player'}):
        a_tag = a.find('a')
        player_links.append(a_tag.get('href'))

    table = soup.find('table', id='player-contracts')
    df = pd.read_html(str(table))[0]

    df['Link'] = player_links
    df['Link'] = df['Link'].apply(lambda x: 'https://www.basketball-reference.com' + x)

    return df

#scrape_contract_data().to_csv('contracts.csv')

In [14]:
def scrape_player_data():
    contracts = pd.read_csv('contracts.csv', header=0)
    player_links = contracts['Link']
    index = 0
    dfs = []

    for link in player_links:
        r = requests.get(link)
        soup = BeautifulSoup(r.text, 'html.parser')

        for header in soup.find_all(class_='over_header'):
            header.decompose()
        
        for thead in soup.find_all(class_='thead'):
            thead.decompose()

        for tfoot in soup.find_all('tfoot'):
            tfoot.decompose()

        name = soup.find('h1', {'itemprop': 'name'}).text

        if soup.find('table', id='totals'):
            table = soup.find('table', id='totals')

            df = pd.read_html(str(table))[0]
            df['Player'] = name
            dfs.append(df)
        else:
            print("Could not load table for", name)
        
        index += 1
        print(index, "of", len(player_links))
    
    return dfs

#dfs = scrape_player_data()

#df = pd.concat(dfs)
#df.to_csv('player_data.csv')